In [1]:
%pip install ollama

  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached h11-0.14.0-py3-none-any.whl.metadata (8.2 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
Using cached httpx-0.28.1-py3-none-any.whl (73 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 10.2 MB/s eta 0:00:00
Using cached annotated_types-0.7.0-py3-none-any.whl (13 kB)
Using cached idna-3.10-py3-none-any.whl (70 kB)
Using cached h11-0.14.0-py3-none-any.whl (58 kB)
Using cached sniffio-1.3.1-py3-none-any.whl (10 kB)
Note: you may need to restart the kernel to use updated packages.


In [ ]:
"""This script evaluates an LLM prompt for processing text so that it can be used for the wttr.in API"""
 
from ollama import Client
 
LLM_MODEL: str = "gemma3:27b"    # Optional, change this to be the model you want
client: Client = Client(
  host='http://10.1.69.214:11434' # Optional, change this to be the URL of your LLM
)
 
 
# TODO: define  llm_parse_for_wttr()
def llm_parse_for_wttr(input_str) -> str:
    #Give the LLM a post
    #Strip the reponse for the result
    llmResponse = client.chat(model=LLM_MODEL, messages=[
            {
                'role': 'user',
                'content': '{input_str}',
            },
        ])
    return llmResponse
 
 
 
# Test cases
test_cases = [ # TODO: Replace these test cases with ones for wttr.in
    {
        "input": "Extract the number: The total is 42 units.",
        "expected": "42"
    },
    {
        "input": "What color is the fruit: banana.'",
        "expected": "Yellow"
    },
    {
        "input": "Give me the JSON: Name is Alice, age is 30",
        "expected": '{"name": "Alice", "age": 30}'
    },
]
 
# Function to iterate through test cases
def run_tests():
    num_passed = 0
 
    for i, test in enumerate(test_cases, 1):
        raw_input = test["input"]
        expected_output = test["expected"]
 
        print(f"\nTest {i}: {raw_input}")
        try:
            result = llm_parse_for_wttr(raw_input).strip()
            expected = expected_output.strip()
 
            print("LLM Output  :", result)
            print("Expected    :", expected)
 
            if result == expected:
                print("✅ PASS")
                num_passed += 1
            else:
                print("❌ FAIL")
 
        except Exception as e:
            print("💥 ERROR:", e)
 
    print(f"\nSummary: {num_passed} / {len(test) + 1} tests passed.")
 
# Run the test cases
run_tests()